In [2]:
import nltk 
import re
from nltk.stem import PorterStemmer
import os 
import json
from charset_normalizer import from_path
STEMMER = PorterStemmer()

## I. Read the files and build two large consolidate files that are the union of all the documents in 20N and BAC.

In [3]:
ACTUAL_PATH = os.getcwd()
PATH_20N = os.path.join(ACTUAL_PATH, "data/20news-18828")
PATH_BAC = os.path.join(ACTUAL_PATH, "data/BAC/blogs")
PATH_FINAL_FILES = os.path.join(ACTUAL_PATH,"data/final_files")
GRUPO = "03"

### UPLOAD_FILES

#### UPLOADING 20N 

In [ ]:
NEW_20N_FILE = os.path.join(PATH_FINAL_FILES, "20N.jsonl")

mayor_folders_20N = os.listdir(PATH_20N)
dictionary = {}

with open(NEW_20N_FILE, "w", encoding="utf-8", errors="replace") as f_n:
    for folder in mayor_folders_20N:
        minor_files_path = os.path.join(PATH_20N, folder)
        minor_files = os.listdir(minor_files_path)
        for file in minor_files:
            file_path = os.path.join(minor_files_path, file)
            with open(file_path, "r", encoding="utf-8", errors="replace") as f:
                text = f.read().strip()

            record = {
                "id": file,
                "theme": folder,
                "source": "20N",
                "text": text
            }
            unit = folder + file
            if file in dictionary.keys():
                dictionary[unit] += 1
            else: 
                dictionary[unit] = 1
            f_n.write(json.dumps(record, ensure_ascii=False) + "\n")

In [ ]:
## TODO: Verificar si los encodings estan bien

#### UPLOADING BAC

In [ ]:
NEW_BAC_FILE = os.path.join(PATH_FINAL_FILES, "BAC.jsonl")
mayor_folders_BAC = os.listdir(PATH_BAC)
with open(NEW_BAC_FILE, "w", encoding="utf-8", errors="replace") as f_n:
    for file in mayor_folders_BAC:
        post_num = 0
        file_path = os.path.join(PATH_BAC, file)
        with open(file_path, "r", encoding="utf-8", errors="replace") as f:
            text = f.read().strip()
            text = re.sub(r"</?Blog>", "", text)
        post_list = text.split("<post>")
        for post in post_list:
            post = post.strip().replace("</post>", "")
            record = {
                "id": file,
                "post_num": post_num,
                "source": "BAC",
                "text": post
            }
            f_n.write(json.dumps(record, ensure_ascii=False) + "\n")
            post_num +=1

## II. Tokenize by sentence

In [4]:
import nltk
import re

In [5]:
# def get_sentences(text:str, k: int) -> list[list]:
#     sentences_list = []
#     text = text.split()
#     last = 0
#     for i in range(k,len(text)+1):
#         sentences = text[last:i]
#         last +=1
#         text_slice = " ".join(sentences)
#         text_slice_token = preprocess_text(text_slice)
#         sentences_list.append(text_slice_token)
#     return sentences_list
def preprocess_text(text:str) -> list[str]:

    text = text.lower() 
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text, flags=re.I | re.A | re.MULTILINE)
    text = re.sub(r"\d", "NUM", text)
    text = "<s> " + text
    text = text + " </s>"
    text = text.strip().split()
    #tokens = [STEMMER.stem(word) for word in text]
    return text
def divide_sentences(text:str)->list[str]:
    sentences = nltk.sent_tokenize(text)
    return sentences

    

### Examples

In [12]:
text = "This is sentence one. Here is another! And number 123."
sentences = divide_sentences(text)
sentences

['This is sentence one.', 'Here is another!', 'And number 123.']

In [13]:
processed = [preprocess_text(s) for s in sentences]

print(processed)

[['<s>', 'this', 'is', 'sentence', 'one', '</s>'], ['<s>', 'here', 'is', 'another', '</s>'], ['<s>', 'and', 'number', 'NUMNUMNUM', '</s>']]


### Tokenizing_full_text

In [26]:
import sys

def deep_getsizeof(obj):
    """Calcula tamaño total de una lista de strings."""
    if isinstance(obj, list):
        return sys.getsizeof(obj) + sum(deep_getsizeof(x) for x in obj)
    elif isinstance(obj, str):
        return sys.getsizeof(obj)
    else:
        return sys.getsizeof(obj)

print(deep_getsizeof(sentencias), "bytes")

417365863 bytes


In [27]:
pip install pympler

Now using node v22.17.0 (npm v10.9.2)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import random
import pickle

def save_pickle(data, filename):
    filepath = os.path.join(PATH_FINAL_FILES, filename)
    with open(filepath, "wb") as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

sentencias = []
with open(os.path.join(PATH_FINAL_FILES, "20N.jsonl"),"r") as f:
    for line in f:
        line = json.loads(line)
        sentences = divide_sentences(line["text"])
        for sentence in sentences:
            pre_process = preprocess_text(sentence)
            sentencias.append(pre_process)
            #print(sentence, pre_process)
random.seed(42)
random.shuffle(sentencias)
index_to_split = int(0.8 * len(sentencias))
train_sentences = sentencias[:index_to_split]
test_sentences = sentencias[index_to_split:]
save_pickle(train_sentences, f"20N_{GRUPO}_training.pkl")
save_pickle(test_sentences, f"20N_{GRUPO}_testing.pkl")

In [ ]:
import random
import pickle

def save_pickle(data, filename):
    filepath = os.path.join(PATH_FINAL_FILES, filename)
    with open(filepath, "wb") as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

sentencias = []
with open(os.path.join(PATH_FINAL_FILES, "BAC.jsonl"),"r") as f:
    for line in f:
        line = json.loads(line)
        sentences = divide_sentences(line["text"])
        for sentence in sentences:
            pre_process = preprocess_text(sentence)
            sentencias.append(pre_process)
            #print(sentence, pre_process)
random.seed(42)
random.shuffle(sentencias)
split_idx = int(0.8 * len(sentencias))
train_sentences = sentencias[:split_idx]
test_sentences = sentencias[split_idx:]
save_pickle(train_sentences, f"BAC_{GRUPO}_training.pkl")
save_pickle(test_sentences, f"BAC_{GRUPO}_testing.pkl")